In [6]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras.layers import Embedding
from keras.utils import np_utils
import numpy as np

Using TensorFlow backend.
/Users/marcosgallo/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/marcosgallo/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/marcosgallo/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/marcosgallo/anaconda3/lib/pytho

In [16]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

raw_text = load_doc('data/char_sequences.txt')
lines = raw_text.split('\n')

chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))

sequences = list()
for line in lines:
    # integer encode line
    encoded_seq = [mapping[char] for char in line]
    sequences.append(encoded_seq)
    
vocab_size = len(mapping)
sequences = array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]

sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)

In [17]:
# Define model
model = Sequential()
model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 75)                34200     
_________________________________________________________________
dense_3 (Dense)              (None, 38)                2888      
Total params: 37,088
Trainable params: 37,088
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
93787/93787 [==============================] - 85s 907us/step - loss: 2.2990 - accuracy: 0.3395
Epoch 2/100
93787/93787 [==============================] - 86s 922us/step - loss: 2.0545 - accuracy: 0.3953
Epoch 3/100
93787/93787 [==============================] - 87s 928us/step - loss: 1.9540 - accuracy: 0.4176
Epoch 4/100
93787/93787 [==============================] - 88s 941us/step - loss: 1.8888 - accuracy: 0.4360
Epoch 5/100
93787/93787 [==============================] - 94s 1ms/step - loss: 1.8385 - accuracy: 0.4493
Epoch 6/100
93787/93787 [==============================] - 87s 926us/step - loss: 1.7986 - accuracy: 0.4599
Epoch 7/100
93787/93787 [==============================] - 81s 868us/step - loss: 1.7659 - accuracy: 0.4686
Epoch 8/100
93787/93787 [==============================] - 84s 898us/step - loss: 1.7374 - accuracy: 0.4758
Epoch 9/100
93787/93787 [==============================] - 85s 905us/step - loss: 1.7136 - accuracy: 0.4818
Epoch 10/100
93787/93787 [====

93787/93787 [==============================] - 86s 920us/step - loss: 1.3413 - accuracy: 0.5785
Epoch 77/100
93787/93787 [==============================] - 85s 905us/step - loss: 1.3387 - accuracy: 0.5780
Epoch 78/100
93787/93787 [==============================] - 85s 910us/step - loss: 1.3373 - accuracy: 0.5785
Epoch 79/100
93787/93787 [==============================] - 84s 899us/step - loss: 1.3347 - accuracy: 0.5789
Epoch 80/100
93787/93787 [==============================] - 85s 905us/step - loss: 1.3323 - accuracy: 0.5792
Epoch 81/100
93787/93787 [==============================] - 85s 911us/step - loss: 1.3306 - accuracy: 0.5790
Epoch 82/100
93787/93787 [==============================] - 86s 912us/step - loss: 1.3274 - accuracy: 0.5811
Epoch 83/100
93787/93787 [==============================] - 85s 903us/step - loss: 1.3255 - accuracy: 0.5812
Epoch 84/100
93787/93787 [==============================] - 85s 907us/step - loss: 1.3229 - accuracy: 0.5819
Epoch 85/100
93787/93787 [======

In [22]:
model.save('model2.h5')
dump(mapping, open('mapping2.pkl', 'wb'))

In [32]:
from keras.models import load_model
from pickle import load
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# one hot encode
		encoded = to_categorical(encoded, num_classes=len(mapping))
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:
				out_char = char
				break
		# append to input
		in_text += char
	return in_text

In [33]:
# load the model
model = load_model('model.h5')
# load the mapping
mapping = load(open('mapping.pkl', 'rb'))

In [40]:
# test start of rhyme
print(generate_seq(model, mapping, 40, "o know sweet love i always write of you, ", 600))

o know sweet love i always write of you, and in my self thou art conscaied with the heart, which i think of men his stars no remember to thee, and then thou art thee to thee, and then thou art thee, and i which it thy self thou art thee, when i cannot be for my self and thee shall bear the store the world to thee the world that i have seem with a have should love thee the worst that thou dost thou thy self thou art confounding, and thou the charter as thou wilt bountess come, and therefore sees shade, that thou dost thou thy self thou art confounding, and thou the charter as thou wilt bountess come, and therefore sees shade, that tho
